In [1]:
import sys
import gym
import numpy as np
from collections import defaultdict

from utils.plot_utils import plot_blackjack_values, plot_policy

Use the code cell below to create an instance of the [Blackjack](https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py) environment.

In [2]:
env = gym.make('Blackjack-v0')

Each state is a 3-tuple of:
- the player's current sum $\in \{0, 1, \ldots, 31\}$,
- the dealer's face up card $\in \{1, \ldots, 10\}$, and
- whether or not the player has a usable ace (`no` $=0$, `yes` $=1$).

The agent has two potential actions:

```
    STICK = 0
    HIT = 1
```
Verify this by running the code cell below.

In [3]:
print(env.observation_space)
print(env.action_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))
Discrete(2)


In [4]:
for i_episode in range(3):
    state = env.reset()
    while True:
        print(state)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if done:
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break

(17, 3, False)
End game! Reward:  -1.0
You lost :(

(10, 9, False)
(12, 9, False)
End game! Reward:  -1.0
You lost :(

(15, 7, False)
End game! Reward:  1.0
You won :)



###  MC Control

In this section, you will write your own implementation of constant-$\alpha$ MC control.  

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.
- `policy`: This is a dictionary where `policy[s]` returns the action that the agent chooses after observing state `s`.

(_Feel free to define additional functions to help you to organize your code._)

In [5]:
def generate_episode(bj_env, policy=None):
    episode = []
    state = bj_env.reset()
    nA = bj_env.action_space.n
    while True:
        if (state) in policy:
            action = policy[state]
        else:
            action = np.random.choice(np.arange(nA))
        
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

In [6]:
def get_discounted_return(gamma, rewards):
    reward = rewards.pop(0)
    if len(rewards) == 0: 
        return reward
    else:
        return(reward + gamma*get_discounted_return(gamma, rewards))

In [7]:
def epsilon_greedy_policy(Q, epsilon):
    policy={}
    for state, action in Q.items():
        best_index = np.argmax(action)
        n_a = len(action)
        
        non_greedy_prob = epsilon/n_a      
        probs=np.ones_like(action)*non_greedy_prob
        
        greedy_prob = 1-epsilon + (epsilon/n_a)
        probs[best_index] = greedy_prob
        
        policy[state]= np.random.choice(np.arange(action.size), p=probs)
        
        
    return policy

In [8]:
def mc_control(env, num_episodes, alpha, gamma=0.9, epsilon=1.0):
    nA = env.action_space.n
    # initialize empty dictionary of arrays
    Q = defaultdict(lambda: np.zeros(nA))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):       
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
           
        epsilon_i = epsilon - ((i_episode-1)/(num_episodes-1))
        policy = epsilon_greedy_policy(Q, epsilon_i)
            
        # generate an episode using new policy
        episode=generate_episode(env, policy)
        
        # Iterate over transitions
        states, actions, rewards = zip(*episode)      
        actions=list(actions)
        rewards=list(rewards)
        
        episode_first_counter={}
        for t_i, state in enumerate(states):
            action = actions.pop(0)           
            # Check if its first visit
            if not ((state, action) in episode_first_counter):
                episode_first_counter[(state, action)] = 1
                
                G_t=get_discounted_return(gamma, rewards.copy())
                Q[state][action] += alpha*(G_t-Q[state][action])

            rewards.pop(0)
            
    return policy, Q

Use the cell below to obtain the estimated optimal policy and action-value function.  Note that you should fill in your own values for the `num_episodes` and `alpha` parameters.

In [ ]:
# obtain the estimated optimal policy and action-value function
policy, Q = mc_control(env, num_episodes=100000, alpha=0.1)

Episode 32000/100000.

Next, we plot the corresponding state-value function.

In [ ]:
# obtain the corresponding state-value function
V = dict((k,np.max(v)) for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V)

Finally, we visualize the policy that is estimated to be optimal.

In [ ]:
# plot the policy
plot_policy(policy)

The **true** optimal policy $\pi_*$ can be found in Figure 5.2 of the [textbook](http://go.udacity.com/rl-textbook) (and appears below).  Compare your final estimate to the optimal policy - how close are you able to get?  If you are not happy with the performance of your algorithm, take the time to tweak the decay rate of $\epsilon$, change the value of $\alpha$, and/or run the algorithm for more episodes to attain better results.

![True Optimal Policy](images/optimal.png)